# Librairies



In [1]:
import pickle
import numpy as np
import os
import cv2
from sklearn.neighbors import KNeighborsClassifier
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Création du dataset : collecte et préparation


In [15]:
# ================== #
# Variables globales #
# ================== #

# Création d'un répertoire data
if not os.path.exists("data"):
    os.mkdir("data")

nb_images_collectees = 10  # Nombre d'image qu'on veut collecter pour chacun des admis et des non admis

donnees_visage = []  # Liste vide pour enregistrer les visages
etiquettes = []  # Liste vide pour enregistrer nos étiquetes "admis" ou "non admis"

# =============================== #
# Capture des data: nom et images #
# =============================== #

# Définition de la fonction pour capturer une photo
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button'); // bouton pour capture d'image
            capture.textContent = 'Capture';
            div.appendChild(capture);
            // permet l'affichage du flux vidéo de notre caméra en temps réel
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            // l'exécution du script s'interromp jusqu'à ce qu'un clic pour la capture soit effectué
            await new Promise((resolve) => capture.onclick = resolve);
            //  capture de l'image, arrêt de la caméra, mise en format jpeg de l'image
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    # On exécute le script JavaScript et on réceptionne l'image capturée sous forme de chaîne de données base64
    data = eval_js('takePhoto({})'.format(quality))
    # on décode la chaîne base64 pour obtenir les données de l'image en binaire
    binary = b64decode(data.split(',')[1])
    # Sauvegarde des données binaires dans un fichier image
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# on charge le modèl qui permet de détecter les visage
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Collecte des images qui sont considéré comme "admis"
print("Collecte des photos 'non admis'")
for i in range(nb_images_collectees):
    print(f"Capture {i+1}/{nb_images_collectees} pour 'admis'")
    filename = take_photo()  # Capture une photo
    image = cv2.imread(filename)
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

    for (x, y, w, h) in coordonnees_visage:
        visage = image[y:y+h, x:x+w]
        visage_redimensionne = cv2.resize(visage, (50, 50))
        donnees_visage.append(visage_redimensionne)
        break  # on sauvegarde que le 1er photo
    etiquettes.append("admis")

# Collecte des images qui sont considéré comme "non admis"
print("Collecte des photos 'non admis'")
for i in range(nb_images_collectees):
    print(f"Capture {i+1}/{nb_images_collectees} pour 'non admis'")
    filename = take_photo()
    image = cv2.imread(filename)
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

    for (x, y, w, h) in coordonnees_visage:
        visage = image[y:y+h, x:x+w]
        visage_redimensionne = cv2.resize(visage, (50, 50))
        donnees_visage.append(visage_redimensionne)
        etiquettes.append("non admis")
        break
# on convertit les données des visages en un tableau numpy
donnees_visage = np.array(donnees_visage).reshape(len(donnees_visage), -1)

Collecte des photos 'non admis'
Capture 1/10 pour 'admis'


<IPython.core.display.Javascript object>

Capture 2/10 pour 'admis'


<IPython.core.display.Javascript object>

Capture 3/10 pour 'admis'


...

<IPython.core.display.Javascript object>

Capture 9/10 pour 'non admis'


<IPython.core.display.Javascript object>

Capture 10/10 pour 'non admis'


<IPython.core.display.Javascript object>

# Souvegarde de données

In [16]:
# Enregistrement des visages et des étiquettes
with open('data/visages.pkl', 'wb') as file:
    pickle.dump(donnees_visage, file)

with open('data/etiquettes.pkl', 'wb') as file:
    pickle.dump(etiquettes, file)

# Algorithme

In [17]:
# Chargement du data
with open('data/visages.pkl', 'rb') as file:
    visages = pickle.load(file)

with open('data/etiquettes.pkl', 'rb') as file:
    etiquettes = pickle.load(file)

In [18]:
# Entrainement du modèl KNN
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(visages, etiquettes)

KNeighborsClassifier(n_neighbors=1)

# Exécution

In [19]:
# Exemple d''exécution
filename_test = take_photo('test_photo.jpg') #prend une nouvelle photo
# Traitement de la photo prise
image_test = cv2.imread(filename_test)
gris_test = cv2.cvtColor(image_test, cv2.COLOR_BGR2GRAY)
coordonnees_visage_test = cascade_visage.detectMultiScale(gris_test, 1.3, 5)

for (x, y, w, h) in coordonnees_visage_test:
    visage_test = image_test[y:y+h, x:x+w]
    visage_test_redimensionne = cv2.resize(visage_test, (50, 50))
    break

visage_test_flatten = visage_test_redimensionne.reshape(1, -1)  # applatit la photo
# Prédiction à l'aide de l'algorithme KNN
prediction = knn.predict(visage_test_flatten)
print(f"Le visage est prédit comme : {'Admis' if prediction[0] == 'admis' else 'Non admis'}")


<IPython.core.display.Javascript object>

Le visage est prédit comme : Non admis
